In [1]:
from pyscf import gto,scf
import pandas as pd
from pyscf.hessian import rhf as hrhf
import numpy as np 
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
import basis_set_exchange as bse
import sys
dl=.05
sys.path.append('/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/')
from FDcoeffs import *
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM

In [2]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x

In [3]:
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [8]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()

converged SCF energy = -40.2151804034984


-40.2151804034984

In [9]:
#ad1n,adn2=alc_deriv_grad_nuc(CH4,[1,0,0,0,0]),alc_differential_grad_nuc(CH4,[1,0,0,0,0])

In [27]:
dl=.1
gs=[]
ge=[]
hs=[]
amds=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,0, 0, 0, 0],atom=ch4str,unit="Bohrs",\
           basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},verbose=0,charge=i*dl)
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e(),max_cycle=100)
    print(mf.mol.nelec)
    g=mf.Gradients()
    hs.append(mf.Hessian().hess()[1,1,2,2])
    gs.append(g.grad()[1,2])
    amds.append((aaff(mf,[1,0,0,0,0])+alc_deriv_grad_nuc(fm,[1,0,0,0,0]))[1,2])

(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)


In [28]:
ch4_gs=np.asarray(gs)
ch4_hs=np.asarray(hs)
ch4_amds=np.asarray(amds)

In [29]:
np.save("Ch4_grads_9pfd",ch4_gs),np.save("Ch4_hess_9pfd",ch4_hs)
np.save("Ch4_aderivs_9pfd",ch4_amds)

In [33]:
print("2 order finite diff. accuracy:: ",np.dot(fd1_2,gs)/dl)
print("4 order finite diff. accuracy:: ",np.dot(fd1_4,gs)/dl)
print("6 order finite diff. accuracy:: ",np.dot(fd1_6,gs)/dl)
#print("8 order finite diff. accuracy:: ",np.dot(fd1_8,gs)/dl)
print("Analytical derivative        :: ",amds[4])

2 order finite diff. accuracy::  0.07884730081702318
4 order finite diff. accuracy::  0.07891979106868469
6 order finite diff. accuracy::  0.07891988016489244
8 order finite diff. accuracy::  0.07891990991545973
Analytical derivative        ::  0.07891936579471134


In [ ]:
2 order finite diff. accuracy::  0.078847
4 order finite diff. accuracy::  0.078920
6 order finite diff. accuracy::  0.078920
Analytical derivative        ::  0.078919

In [14]:
NH4=FcM(fcs=[1,0,0,0,0],atom=ch4str,unit="bohr",\
          basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
NH4.nelectron=10

In [15]:
hfnh4=scf.RHF(NH4)
hfnh4.scf(dm0=hfnh4.init_guess_by_1e())

converged SCF energy = -56.5506646111419


-56.55066461114188

In [16]:
NH4.nelec

(5, 5)

In [17]:
gnh4,gnh4n=hfnh4.Gradients().grad()[1,2],hfnh4.Gradients().grad_nuc()[1,2]

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000    -0.0000000000
1 H     0.0000000000    -0.0000000000     0.0534595930
2 H     0.0000000000     0.0504021876    -0.0178198643
3 H     0.0436495749    -0.0252010938    -0.0178198643
4 H    -0.0436495749    -0.0252010938    -0.0178198643
----------------------------------------------


In [18]:
gnh4n

-1.8955982396089313

In [20]:
mf.Gradients().grad_nuc()

array([[ 0.00000000e+00, -4.44089210e-16, -6.66133815e-16],
       [ 0.00000000e+00, -6.93889390e-18, -1.70408882e+00],
       [ 0.00000000e+00, -1.60663034e+00,  5.68029606e-01],
       [-1.39138269e+00,  8.03315172e-01,  5.68029606e-01],
       [ 1.39138269e+00,  8.03315172e-01,  5.68029606e-01]])

In [21]:
1/2.043854241378284**2

0.23938677763795033

In [22]:
hnh4=hfnh4.Hessian().hess()

In [23]:
gnh4,gnh4n,hnh4[1,1,2,2]

(0.05345959300969616, -1.8955982396089313, 0.3036046690171408)

In [24]:
BH4=FcM(fcs=[-1,0,0,0,0],atom=ch4str,unit="bohr",\
          basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
BH4.nelectron=10
hfbh4=scf.RHF(BH4)
hfbh4.scf(dm0=hfbh4.init_guess_by_1e())

converged SCF energy = -26.9391542012826


-26.93915420128262

In [25]:
gbh4,gbh4n=hfbh4.Gradients().grad()[1,2],hfbh4.Gradients().grad_nuc()[1,2]

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.0000000000
1 H     0.0000000000    -0.0000000000    -0.0900956221
2 H     0.0000000000    -0.0849429671     0.0300318740
3 H    -0.0735627674     0.0424714836     0.0300318740
4 H     0.0735627674     0.0424714836     0.0300318740
----------------------------------------------


In [26]:
hbh4=hfbh4.Hessian().hess()

In [27]:
gbh4,gbh4n,hbh4[1,1,2,2]

(-0.0900956220918725, -1.4168246843330305, 0.40295756361548374)

In [28]:
BH4.nelectron

10

In [29]:
alc_deriv_grad_nuc(CH4,[1,0,0,0,0]),alc_differential_grad_nuc(CH4,[1,0,0,0,0]),\
alc_deriv_grad_nuc(CH4,[-1,0,0,0,0]),alc_differential_grad_nuc(CH4,[-1,0,0,0,0])

(array([[ 0.00000000e+00, -1.11022302e-16, -5.55111512e-17],
        [ 0.00000000e+00,  0.00000000e+00, -2.39386778e-01],
        [ 0.00000000e+00, -2.25696018e-01,  7.97955925e-02],
        [-1.95458485e-01,  1.12848009e-01,  7.97955925e-02],
        [ 1.95458485e-01,  1.12848009e-01,  7.97955925e-02]]),
 array([[ 0.00000000e+00, -1.11022302e-16, -5.55111512e-17],
        [ 0.00000000e+00,  0.00000000e+00, -2.39386778e-01],
        [ 0.00000000e+00, -2.25696018e-01,  7.97955925e-02],
        [-1.95458485e-01,  1.12848009e-01,  7.97955925e-02],
        [ 1.95458485e-01,  1.12848009e-01,  7.97955925e-02]]),
 array([[ 0.00000000e+00,  1.11022302e-16,  5.55111512e-17],
        [ 0.00000000e+00,  0.00000000e+00,  2.39386778e-01],
        [ 0.00000000e+00,  2.25696018e-01, -7.97955925e-02],
        [ 1.95458485e-01, -1.12848009e-01, -7.97955925e-02],
        [-1.95458485e-01, -1.12848009e-01, -7.97955925e-02]]),
 array([[ 0.00000000e+00,  1.11022302e-16,  5.55111512e-17],
        [ 0.000000

In [30]:
1/z

0.4892716808052049